Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import cobra
import escher

# Load model

In [ ]:
model = cobra.io.read_sbml_model("./models/yeast-GEM-BiGG.xml")
model

# Inspect functions, metabolites, and reactions

Objective function is `GROWTH` in the yeast-GEM model, inspecting it and related reactions:

In [ ]:
model.reactions.get_by_id('GROWTH')

In [ ]:
model.metabolites.get_by_id('biomass_c')

In [ ]:
# This is the biomass pseudoreaction -- we're interested in the stoichiometry of this.
biomass = model.reactions.get_by_id('BIOMASS_yeast-GEM')
biomass
# Conveniently, it has one 'metabolite' for all lipids, one for all proteins,
# one for RNA, etc.  We want to remove each in turn, so if it's written in this way,
# it's super easy to do so.

Inspecting each type of macromolecule...

Lipid

In [ ]:
model.metabolites.get_by_id('s_1096')

In [ ]:
# Lipid pseudoreaction is combining 'bulk' lipid backbone and 'bulk' lipid chain substrates
model.reactions.get_by_id('BIOMASS_yeast-GEM_LIP')

Protein

In [ ]:
model.metabolites.get_by_id('protein_c')

In [ ]:
# Protein pseudoreaction is combining aminoacyl-tRNAs
model.reactions.get_by_id('BIOMASS_yeast-GEM_PROT')

DNA

In [ ]:
model.metabolites.get_by_id('dna_c')

In [ ]:
# DNA pseudoreaction is combining dNTPs
model.reactions.get_by_id('BIOMASS_yeast-GEM_DNA')

RNA

In [ ]:
model.metabolites.get_by_id('rna_c')

In [ ]:
# RNA pseudoreaction is combining NTPs
model.reactions.get_by_id('BIOMASS_yeast-GEM_RNA')

# Unmodified model

Limit glucose uptake (minimal media)

In [ ]:
model.reactions.EX_glc__D_e.bounds = (-18.5, 1) # TODO, find a sensible number

Simulate model and draw fluxes through central carbon metabolism

In [ ]:
solution = model.optimize()
b = escher.Builder(
    map_name='iMM904.Central carbon metabolism',
    reaction_data=solution.fluxes.to_dict()
)
b

Get flux through biomass reaction

In [ ]:
biomass = model.reactions.get_by_id('BIOMASS_yeast-GEM')
print(f'Flux through biomass reaction is {biomass.flux:.4f} mmol/(gDW h)')

Estimate timescale for biomass synthesis

In [ ]:
# Define constants
CELL_DRY_MASS = 15e-12 # g
MOLWEIGHT_BIOMASS = 0.966 # g/mmol, Takhaveev et al. (2023)

In [ ]:
biomass_time = 1/(biomass.flux * MOLWEIGHT_BIOMASS)
print(f'Estimated time: {biomass_time:.4f} hours')

## Modify biomass reaction by ablating each type of macromolecule

In [ ]:
CELL_DRY_MASS = 15e-12 # g

class BiomassComponent():
    def __init__(
        self,
        metabolite_label,
        metabolite_id,
        molecular_mass,
        mass_per_cell,
    ):
        self.metabolite_label = metabolite_label
        self.metabolite_id = metabolite_id
        self.molecular_mass = molecular_mass
        self.mass_per_cell = mass_per_cell
        
        self.ablated_flux = None
        self.est_time = None
        
    def get_est_time(self):
        self.est_time = self.mass_per_cell/(CELL_DRY_MASS * self.ablated_flux * self.molecular_mass)
        

In [ ]:
model_saved = cobra.io.read_sbml_model("./models/yeast-GEM-BiGG.xml")

In [ ]:
# TODO:
# - Create CSV table containing these
# - Create a class builder that builds these classes based on the CSV table
# - FURTHER: make it able to deal with ranges of values (lower limit, upper limit)

Lipids = BiomassComponent(
    metabolite_label='lipid',
    metabolite_id='s_1096',
    molecular_mass=0.800,
    mass_per_cell=900e-15,
)

Proteins = BiomassComponent(
    metabolite_label='protein',
    metabolite_id='protein_c',
    molecular_mass=55,
    mass_per_cell=7650e-15,
)

Carbohydrates = BiomassComponent(
    metabolite_label='carbohydrate',
    metabolite_id='s_3718',
    molecular_mass=1000, # 'variable'
    mass_per_cell=(75+3450)*1e-15, # 'storage carbohydrates' + 'structural polymers'
)

DNA = BiomassComponent(
    metabolite_label='DNA',
    metabolite_id='dna_c',
    molecular_mass=2.5e5,
    mass_per_cell=75e-15,
)

RNA = BiomassComponent(
    metabolite_label='RNA',
    metabolite_id='rna_c',
    molecular_mass=1e2, # 1e4 -- 1e6 Da
    mass_per_cell=1650e-15,
)

In [ ]:
biomass_component_list = [Lipids, Proteins, Carbohydrates, DNA, RNA]

all_metabolite_ids = [
    biomass_component.metabolite_id
    for biomass_component in biomass_component_list
]

for biomass_component in biomass_component_list:
    # get model
    model = model_saved.copy()
    # model.reactions.EX_glc__D_e.bounds = (-18.5, 1) # Limit glucose uptake.  TODO, find a sensible number
    biomass_reaction = model.reactions.get_by_id('BIOMASS_yeast-GEM')
    
    # boilerplate: lookup
    to_ablate = all_metabolite_ids.copy()
    to_ablate.remove(biomass_component.metabolite_id)
    to_ablate_keys = [
        model.metabolites.get_by_id(metabolite_id)
        for metabolite_id in to_ablate
    ]
    to_ablate_dict = dict(zip(to_ablate_keys, [-1]*len(to_ablate_keys)))
    
    # ablate metabolites from biomass reaction
    biomass_reaction.subtract_metabolites(to_ablate_dict)
    
    # optimise model
    solution = model.optimize()
    biomass_component.ablated_flux = biomass_reaction.flux
    biomass_component.get_est_time()
    print(biomass_component.metabolite_label)
    print(biomass_reaction)
    print(f'flux through ablated biomass reaction: {biomass_component.ablated_flux:.4f} mmol/(gDW h)')
    print(f'estimated time: {biomass_component.est_time:.4f} hours')
    print('\n')
    
total_time = sum([biomass_component.est_time for biomass_component in biomass_component_list])
print(f'sum of times: {total_time:.4f} hours')

Draw fluxes through central carbon metabolism

In [ ]:
b = escher.Builder(
    map_name='iMM904.Central carbon metabolism',
    reaction_data=solution.fluxes.to_dict()
)
b